In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


from data import get_data
from time import time
import joblib


## Get Data

In [2]:
urls = 'data/hs_code.xlsx'
types = 'section'

sheets = ['8_digit','6_digit','4_digit','2_digit', 'test_01', 'Declaration_2019_10']

df = get_data(urls,sheets,types)
print(len(df))
df.sample(10)

Load dataset time:  106.675s
49137


,target,data
11077,2,"spices; vanilla, crushed or ground"
28326,18,straps/bands/bracelets of tex. mat. or base me...
41704,15,aluminium carport exterior 8rdr53sc
30395,15,aluminium carport exterior 8rdb23sc
27417,16,"color video monitors w/flat panel screen, vide..."
5049,11,fabrics; warp knit (including those made on ga...
21180,6,"coloring preparations based on iron oxides, as..."
6112,15,"iron or nonalloy steel; flatrolled, width less..."
41610,15,aluminium window frame g03611labae
7899,16,machinetools; presses for working metal or met...


In [3]:
#save the DataFrame
print("Save DataFrame")

joblib.dump(df, 'data/df_master_section.pkl', compress=1)
print('success')

Save DataFrame
success


##  Create Model

In [2]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\n", "", string)    
    string = re.sub(r"\r", "", string) 
    string = re.sub(r"[0-9]", "digit", string)
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [3]:
import re 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

def create_model(df,names):
    X = []
    for i in range(df.shape[0]):
        X.append(clean_str(df.iloc[i][1]))
    y = np.array(df["target"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5)



    print("pipeline of feature engineering and model")
    model = Pipeline([('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

    print("paramater selection")


    parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}

    print("Training: ")
   
    t0 = time()
    gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
    gs_clf_svm = gs_clf_svm.fit(X, y)
    print(gs_clf_svm.best_score_)
    print(gs_clf_svm.best_params_)

    model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

#fit model with training data
    model.fit(X_train, y_train)

    pred = model.predict(X_test)

    print(accuracy_score(y_test, pred))

#save the model
    print("Save Model")

    joblib.dump(model, 'data/model_'+names +'.pkl', compress=1)


    print (classification_report(y_test, pred))

In [7]:
#['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
 #      '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']

lst =[ '01', '02', '03']

for x in lst:
    names = x
    df = joblib.load('data/df_master_section.pkl')
    df['target'].map('{:02}'.format)
    df[df['target'] == f'{x}']
    
    #save the DataFrame
    
    joblib.dump(df, 'data/df_master_'+x+'.pkl', compress=1)
    print("Save DataFrame")   
    create_model(df,names)

ValueError: '=' alignment not allowed in string format specifier

In [12]:
import joblib
print("Load Model")
model = joblib.load('data/model_chapter.pkl')
products = input()  # Video game consoles : 95

Load Model
Iron pyrites


In [13]:
model.predict([products])[0]  # Vanilla : 02

'5'